In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis = 1)


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
#Increasing cutoff to 600
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts() < 600
application_types_to_replace = application_types_to_replace[application_types_to_replace].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

#Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [5]:
#Increasing to 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts() <= 1000
classifications_to_replace = classifications_to_replace[classifications_to_replace].index.tolist()

# Replace in dataframe
for classif in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classif,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_df = pd.get_dummies(application_df, columns = ['CLASSIFICATION','INCOME_AMT','SPECIAL_CONSIDERATIONS','APPLICATION_TYPE','AFFILIATION','USE_CASE',"ORGANIZATION"])
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,INCOME_AMT_0,...,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1,5000,1,1,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,1,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,1,142590,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [7]:
# Split our preprocessed data into our features and target arrays, then train/test
X_train, X_test, y_train, y_test = train_test_split(encoded_df.drop(["IS_SUCCESSFUL"], axis=1 ), 
                                                    encoded_df["IS_SUCCESSFUL"], 
                                                    test_size = 0.7, 
                                                    random_state = 0)



In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
#Adding layers and increasing layer size
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=36, activation="relu", input_dim=42))
nn.add(tf.keras.layers.Dense(units=18, activation='relu', input_dim = 36))
nn.add(tf.keras.layers.Dense(units=48, activation='relu', input_dim = 18))
nn.add(tf.keras.layers.Dense(units=60, activation='relu', input_dim = 48))
nn.add(tf.keras.layers.Dense(units=30, activation='relu', input_dim = 60))
nn.add(tf.keras.layers.Dense(units=8, activation='relu', input_dim = 30))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim=8))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 36)                1548      
                                                                 
 dense_20 (Dense)            (None, 18)                666       
                                                                 
 dense_21 (Dense)            (None, 48)                912       
                                                                 
 dense_22 (Dense)            (None, 60)                2940      
                                                                 
 dense_23 (Dense)            (None, 30)                1830      
                                                                 
 dense_24 (Dense)            (None, 8)                 248       
                                                                 
 dense_25 (Dense)            (None, 1)                

In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
checkpoint_path = "./training/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                save_freq = 'epoch',
                                                period = 5)

fit_model = nn.fit(X_train_scaled, y_train, epochs=30, validation_data = (X_test, y_test), callbacks = cp_callback)

Epoch 1/30
322/322 [==============================] - 1s 2ms/step - loss: 0.5956 - accuracy: 0.6962 - val_loss: 14747.2061 - val_accuracy: 0.5319
Epoch 2/30
322/322 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7335 - val_loss: 2727.9368 - val_accuracy: 0.4681
Epoch 3/30
322/322 [==============================] - 1s 2ms/step - loss: 0.5446 - accuracy: 0.7387 - val_loss: 23816.4980 - val_accuracy: 0.4681
Epoch 4/30
322/322 [==============================] - 1s 2ms/step - loss: 0.5433 - accuracy: 0.7388 - val_loss: 20229.5586 - val_accuracy: 0.4681
Epoch 5/30
277/322 [========================>.....] - ETA: 0s - loss: 0.5396 - accuracy: 0.7411
Epoch 5: saving model to ./training\cp.ckpt
322/322 [==============================] - 1s 2ms/step - loss: 0.5406 - accuracy: 0.7398 - val_loss: 61243.8164 - val_accuracy: 0.4681
Epoch 6/30
322/322 [==============================] - 1s 2ms/step - loss: 0.5393 - accuracy: 0.7409 - val_loss: 79183.2656 - val_accuracy: 0.4

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

751/751 - 0s - loss: 0.5691 - accuracy: 0.7234 - 447ms/epoch - 595us/step
Loss: 0.5690605640411377, Accuracy: 0.7233652472496033


In [24]:
# Export our model to HDF5 file
nn.save(filepath = "model_optimised.h5")